In [ ]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
from datasets import Dataset
import numpy as np

In [ ]:
df = pd.read_csv('df = pd.read_csv('dataset_train.tsv',sep='\t',header=None,names=['phrase', 'target'])dataset_train.tsv',sep='\t',header=None,names=['phrase', 'target'])

In [ ]:
df.groupby(by=['target']).count().sort_values(by=['phrase'],ascending=False)

,phrase
target,
sched_teacher,1110
sched_for_group,405
sched_for_group_day,402
wifi,287
status_free,271
...,...
smalltalk_talk,23
nsu_foundation_date,22
student_trade_union_enter,22


In [ ]:
df = Dataset.from_pandas(df)

In [ ]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
embedding = get_embeddings(df["phrase"][0
                                       ])
embedding.shape

TensorShape([1, 768])

In [ ]:
embeddings_dataset = df.map(
    lambda x: {"embeddings": get_embeddings(x["phrase"]).numpy()[0]}
)

INFO:tensorflow:Assets written to: ram://e8f7d8a8-ec91-4fe7-8eeb-d7a23fcc4bc8/assets


INFO:tensorflow:Assets written to: ram://e8f7d8a8-ec91-4fe7-8eeb-d7a23fcc4bc8/assets


Map:   0%|          | 0/13230 [00:00<?, ? examples/s]

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/14 [00:00<?, ?it/s]

Dataset({
    features: ['phrase', 'target', 'embeddings'],
    num_rows: 13230
})

In [ ]:
test = pd.read_csv('dataset_test.tsv',sep='\t',header=None,names=['phrase', 'target'])

In [ ]:
question_embedding = get_embeddings(test['phrase'].tolist()[0]).numpy()
question_embedding.shape

(1, 768)

In [ ]:
samples['target'][0]

'statement_general'

In [ ]:
def find_nearest(x):
    question_embedding = get_embeddings(x).numpy()
    scores, samples = embeddings_dataset.get_nearest_examples(
        "embeddings", question_embedding, k=1
    )
    return samples['target'][0]

In [ ]:
test['pred'] = test['phrase'].map(find_nearest)

In [ ]:
np.sum((test['pred']==test['target']).astype(int))/len(test)

0.8969422423556059

In [ ]:
df = pd.read_csv('dataset_train.tsv',sep='\t',header=None,names=['phrase', 'target'])

In [ ]:
df['embeddings'] = df.apply(
    lambda x: get_embeddings(x["phrase"]).numpy()[0],axis=1
)

In [ ]:
distilbert_index = nmslib.init(method='hnsw', space='cosinesimil')

In [ ]:
embeddings = df['embeddings'].tolist()
distilbert_index.addDataPointBatch(embeddings)
distilbert_index.createIndex({'post': 2}, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
*

In [ ]:
def search(dataframe, userQuery):

    if dataframe is not None and userQuery is not None:
        df1 = dataframe.copy()
        query = get_embeddings([userQuery])
        ids, distances = distilbert_index.knnQuery(query, k=1)
    matches = []
    for i, j in zip(ids, distances):
        return df1.loc[i]


In [ ]:
pred = test.apply(lambda x: search(df,x['phrase']),axis=1)

In [ ]:
test['pred_1'] = pred['target']

In [ ]:
np.sum((test['pred_1']==test['target']).astype(int))/len(test)

0.8958097395243488